In [1]:
import pandas as pd
import sklearn
import sklearn.model_selection
from sklearn.ensemble import RandomForestClassifier
import sys
sys.path.append('../')
import encoding
from utilities import *
import torch

In [2]:
df = pd.read_csv("../datasets/rollouts.csv")
df

,FEN,Winner
0,rnbqkbnr/pppppppp/8/8/8/5P2/PPPPP1PP/RNBQKBNR ...,0
1,rnbqkbnr/pp1ppppp/8/2p5/8/5P2/PPPPP1PP/RNBQKBN...,0
2,rnbqkbnr/pp1ppppp/8/2p5/8/5P1N/PPPPP1PP/RNBQKB...,0
3,rnbqkbnr/pp1p1ppp/4p3/2p5/8/5P1N/PPPPP1PP/RNBQ...,0
4,rnbqkbnr/pp1p1ppp/4p3/2p5/3P4/5P1N/PPP1P1PP/RN...,0
...,...,...
99995,rn3knr/p1q3pp/1p1p1p2/1bpBQ3/1b2P2R/2P2K2/PB1P...,0
99996,rn3knr/p3q1pp/1p1p1p2/1bpBQ3/1b2P2R/2P2K2/PB1P...,1
99997,rn3knr/p3q1pp/1p1p1p2/1bpBQ3/1b2P3/2P2K2/PB1P1...,0
99998,rn3knr/p2q2pp/1p1p1p2/1bpBQ3/1b2P3/2P2K2/PB1P1...,1


In [ ]:
def error(preds, real):
	return np.sum(np.abs(real.ravel() - preds)) / len(real)

# Advantage encoding

In [ ]:
# Encode the dataset
chess_df = easy_encode(df, encoding.advantage)
chess_df

In [ ]:
train_df, test_df = sklearn.model_selection.train_test_split(chess_df, test_size=.1)
trainX, trainY = XYsplit(train_df, 'Winner')
testX, testY = XYsplit(test_df, 'Winner')

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(trainX, trainY.ravel())
preds = clf.predict(testX)
print(error(preds, testY))

# Binary encoding

In [3]:
# Encode the dataset
chess_df = easy_encode(df, encoding.binary)
chess_df

 59%|█████▉    | 59145/100000 [00:42<00:32, 1262.15it/s]

KeyboardInterrupt: 

In [ ]:
train_df, test_df = sklearn.model_selection.train_test_split(chess_df, test_size=.1)
trainX, trainY = XYsplit(train_df, 'Winner')
testX, testY = XYsplit(test_df, 'Winner')

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(trainX, trainY.ravel())
preds = clf.predict(testX)
print(error(preds, testY))

0.1733


In [ ]:
U, S, V = torch.pca_lowrank(torch.from_numpy(chess_df.iloc[:,:chess_df.columns.size-1].to_numpy()), 2)
chess_df = pd.concat([pd.DataFrame(chess_df.iloc[:,:chess_df.columns.size-1].to_numpy() @ V[:, :2].numpy()), chess_df.iloc[:,chess_df.columns.size-1]], axis=1)